# Plan
- Mix different noise recording + sentence recording as noises
- Sine waves at varying frequency as the signal 
    - random weighting for different harmonics (catch all)
        - complete random for now 
    - custom filter later on 
- varying mixing ratio over time
